# STATISTICAL MODELLING

### Assume that the claims count random variable N has a Poisson distribution with given years at risk v > 0 and expected frequency λ > 0. We aim at modeling the expected frequency λ > 0 such that it allows us to incorporate structural differences (heterogeneity) or systematic effects, between different insurance policies and risks.
### v measures the volume of the aggregated portfolio. Aggregation property says that the aggregated portfolio has a compound Poisson distribution (with volume weighted expected frequency)

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from patsy import dmatrices
import seaborn as sns
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse

In [81]:
claimsdf = pd.read_csv('/home/julian/Cursos/Ironhack/Proyectos/ProyectoFinal/Claims-Frequency-Predictions/Notebooks/claimsdf_1.csv')

In [82]:
claimsdf.head()

,ClaimNb,Exposure,Area,BonusMalus,VehBrand,Region,empirical_frequencies,VehGas_Regular,VehPower_,VehAge_,DrivAge_,log_density
0,1,0.10,4,50,9,1,10.000000,1,5,1,6,7.104144
1,1,0.77,4,50,9,1,1.298701,1,5,1,6,7.104144
2,1,0.75,2,50,9,5,1.333333,0,6,2,6,3.988984
3,1,0.09,2,50,9,7,11.111111,0,7,1,5,4.330733
4,1,0.84,2,50,9,7,1.190476,0,7,1,5,4.330733


In [83]:
claimsdf.drop(columns=['empirical_frequencies'], inplace=True)

## Poisson-GENERALIZED LINEAR MODEL

- The feature components interact in a multiplicative way in our Poisson GLM. One of the main tasks is to analyze whether this multiplicative interaction is appropriate. For GLM modeling approach, as the frequencies are non-linearly related to Vehicle Age and Driver Age as we've seen in the EDA, we should partition them and then treat them as categorical variables.
- We consider 3 continuous feature components (Area, BonusMalus, log-Density), 1 binary feature component (VehGas) and 5 categorical feature components (VehPower, VehAge, DrivAge, VehBrand, Region)
- We'll dummy-encode the categorical features, in order to get a unique MLE for β
- In total, we'll get a 42 variable model.

In [84]:
claimsdf = pd.get_dummies(claimsdf, columns=['VehPower_', 'VehAge_', 'DrivAge_', 'VehBrand', 'Region'], drop_first=True)

In [85]:
claimsdf.head(2)

,ClaimNb,Exposure,Area,BonusMalus,VehGas_Regular,log_density,VehPower__5,VehPower__6,VehPower__7,VehPower__8,...,Region_4,Region_5,Region_6,Region_7,Region_8,Region_9,Region_10,Region_11,Region_12,Region_13
0,1,0.10,4,50,1,7.104144,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0.77,4,50,1,7.104144,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


- REFERENCE LEVEL (Variables for wich the b0 parameter accounts for): `VehPower__4`, `VehAge__1`, `DrivAge__1`, `VehBrand_1`, `Region_1.0`

### MODEL TRAINING

- Maximizing the log-likelihood for parameter β is equivalent to minimizing the deviance loss for β. In this spirit, the deviance loss plays the role of the canonical objective function that should be minimized. b is the scaled Poisson deviance loss. 

- We randomly (uniformly) select 80% of data for training and leave 20% for testing:

In [117]:
sample = np.random.rand(len(claimsdf)) < 0.8

In [118]:
claimsdf_train = claimsdf[sample]

In [119]:
claimsdf_train.shape

(542280, 45)

In [120]:
claimsdf_test = claimsdf[~sample]

In [121]:
claimsdf_test.shape

(135733, 45)

In [122]:
expr = """ Q('ClaimNb') ~ Q('Area') + Q('BonusMalus') + Q('log_density') + Q('VehGas_Regular') + 
                          Q('VehPower__5') + Q('VehPower__6') + Q('VehPower__7') + Q('VehPower__8') +
                          Q('VehPower__9') + Q('VehPower__10') + Q('VehPower__11') + Q('VehPower__12') + 
                          Q('VehAge__2') + Q('VehAge__3') + Q('DrivAge__2') + Q('DrivAge__3') + 
                          Q('DrivAge__4') + Q('DrivAge__5') + Q('DrivAge__6') + Q('DrivAge__7') + Q('DrivAge__8') +
                          Q('VehBrand_2') + Q('VehBrand_3') + Q('VehBrand_4') + Q('VehBrand_5') + Q('VehBrand_6') +
                          Q('VehBrand_7') + Q('VehBrand_8') + Q('VehBrand_9') + Q('VehBrand_10') + Q('VehBrand_11') +
                          Q('Region_2') + Q('Region_3') + Q('Region_4') + Q('Region_5') + Q('Region_6') + 
                          Q('Region_7') + Q('Region_8') + Q('Region_9') + Q('Region_10') + Q('Region_11') +
                          Q('Region_12') """

- Build the matrices for the specified model:

In [123]:
y_train, X_train = dmatrices(expr, claimsdf_train, return_type='dataframe')

- Fit a Poisson-GLM to the TRAIN SET

In [124]:
poisson_model1 = sm.GLM(y_train, X_train, exposure=claimsdf_train.Exposure, family=sm.families.Poisson()).fit()

In [126]:
print(poisson_model1.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:           Q('ClaimNb')   No. Observations:               542280
Model:                            GLM   Df Residuals:                   542237
Model Family:                 Poisson   Df Model:                           42
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.1423e+05
Date:                Tue, 05 Oct 2021   Deviance:                   1.7282e+05
Time:                        11:30:43   Pearson chi2:                 1.34e+06
No. Iterations:                     7                                         
Covariance Type:            nonrobust                                         
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -3.4274    

- From the results of the `poisson_model1` we can see that the variables: `Area`, `VehPower__12`, `VehBrand_2`, `VehBrand_3`, `VehBrand_4`, `VehBrand_6`, `VehBrand_7`, `VehBrand_8`, `VehBrand_10`, `VehBrand_11`, `Region_12`: have a p-value bigger than 0.05 (for an assumed alpha=0.05), so we should consider its inclusion in the model, because they doesn't seem to be significant for frequency modelling.
- As we saw in the correlation analysis, `Area` is highly correlated with `Density`, wich might explain why the first is not significant.
- From the 11 `VehBrand` classes, 8 resulted non-siginificant.

#### DISPERSION:

- Deviance statistics that accounts for potential over- or under-dispersion (φ != 1). In the Poisson model, by definition variance equal to mean (φ = 1). We can determine this parameter empirically by Pearson’s (distribution-free) dispersion estimate and and the deviance dispersion.
- From the results of the `poisson_model1` we can see that the `scale` = 1, wich accounts for the Pearson's dispertion (Pearson's residuals/Residuals degrees of freedom), wich means that we can assume that the mean and variance are equal, the model is not overdispersed. 

- DEVIANCE DISPERTION:

In [96]:
sum((poisson_model1.resid_deviance) ** 2) / 542395

0.31778833768117626

In [97]:
(sum((poisson_model1.resid_pearson) ** 2)) / 542395

2.4360906512514946

#### TRAINING DEVIANCE-LOSS:

In [98]:
loss_train = sum((poisson_model1.resid_deviance) ** 2) / X_train.shape[0]
loss_train

0.31780532964194214

#### AKAIKE INFORMATION CRITERION
- Akaike’s information criterion (AIC), which introduces a penalty term for over-fitting (to mimic an out-of-sample loss)

In [127]:
poisson_model1.aic

228541.37540844904

#### X2-STATISTIC: NULL DEVIANCE - RESIDUAL DEVIANCE:

In [100]:
x2_statistic = (poisson_model1.null_deviance) - sum(poisson_model1.resid_deviance)
x2_statistic

274814.06868109666

#### ROOT MEAN SQUARE ERROR OF PREDICTION:

In [125]:
y_test, X_test = dmatrices(expr, claimsdf_test, return_type='dataframe')

In [131]:
poisson_model1_pred = poisson_model1.predict(X_test)

- Let's create a dataframe with the predctions of the Poisson GLM:

In [150]:
poisson_pred_df = pd.DataFrame(y_test)
poisson_pred_df['predicted_ClaimNb'] = poisson_model1_pred

In [178]:
rmse = []

for _ in range(10):
    p = poisson_pred_df.sample(n=135733)
    rmse_ = ((((p["Q('ClaimNb')"] - p['predicted_ClaimNb']) ** 2).sum())/13574) ** 0.5
    rmse.append(rmse_)

In [188]:
poisson_rmse = sum(rmse)/len(rmse)
poisson_rmse

0.7850454180767814

## Zero Inflated Poisson-GENERALIZED LINEAR MODEL

In [192]:
zip_model = sm.ZeroInflatedPoisson(endog=y_train, exog=X_train, exog_infl=X_train, exposure=claimsdf_train.Exposure, inflation='logit').fit()

/home/julian/miniconda3/envs/ironcon/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


         Current function value: 0.210410
         Iterations: 35
         Function evaluations: 41
         Gradient evaluations: 41


/home/julian/miniconda3/envs/ironcon/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [193]:
print(zip_model.summary())

                     ZeroInflatedPoisson Regression Results                    
Dep. Variable:            Q('ClaimNb')   No. Observations:               542280
Model:             ZeroInflatedPoisson   Df Residuals:                   542237
Method:                            MLE   Df Model:                           42
Date:                 Tue, 05 Oct 2021   Pseudo R-squ.:                 0.02625
Time:                         12:17:18   Log-Likelihood:            -1.1410e+05
converged:                       False   LL-Null:                   -1.1718e+05
Covariance Type:             nonrobust   LLR p-value:                     0.000
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
inflate_Intercept               0.4016      0.231      1.735      0.083      -0.052       0.855
inflate_Q('Area')              -0.0091      0.041     -0.222      0.824 

- From the results of the `zip_poisson_model` we can see that the next variables have a p-value bigger than 0.05 (for an assumed alpha=0.05), so we should consider its inclusion in the model:

- ZERO-INFLATION VARIABLES: The ones that the Logistic regression part of the ZIP model did't find useful for estimating the ϕ parameter:

  `Intercept`, `Area`, `log_density` 
  
  `VehPower_5`, `VehPower_6`, `VehPower__7`, `VehPower_9`, `VehPower_10`, `VehPower_11`,  `VehPower_12`
  
  `DrivAge_2`, `DrivAge_3`, `DrivAge_4`, `DrivAge_5`, `DrivAge_6`, `DrivAge_7`, `DrivAge_8`
  
  `VehBrand_3`, `VehBrand_4`, `VehBrand_5`, `VehBrand_6`, `VehBrand_7`, `VehBrand_8`, `VehBrand_9`, `VehBrand_10`, 
  `VehBrand_11` 
  
  `Region_2`, `Region_3`, `Region_4`, `Region_5`, `Region_6`, `Region_9`, `Region_10`, `Region_11`
  

- VARIABLES did't find useful by the Poisson part of the ZIP model to estimate ClaimNb:

   `VehBrand_3`, `VehBrand_4`, `VehBrand_5`, `VehBrand_7`, `VehBrand_8`, `VehBrand_9`, `VehBrand_10`,       
   `VehBrand_11`
    
   `Region_2`, `Region_3`, `Region_6`, `Region_9`, `Region_10`, `Region_11`
 
   `DrivAge_5`, `DrivAge_6`, `DrivAge_7`, `DrivAge_8`
 
   `VehPower__7`, `VehPower_11`,  `VehPower_12`
 

- For the logistic part, we'll cut: 
  `Intercept`, `Area`, `log_density`, `DriveAge` (rejected 7 out 8 classes), `Vehicle Brand` (rejected 9 of 11 classes), `Region` (rejected 8 out of 13), and `Veh_Power` (7 out of 9 classes)

- For the Poisson part, we'll cut only `Vehicle Brand` (rejected 8 of 11 classes), wich is coincident with the results of the full Poisson model. We'll keep `Drivers Age`, although the rejected classes are the ones that account for the less observed frequencies. With respect to `Region`, we'll keep it because some of the rejected classes account for higer observe frequencies values, maybe the rejection come from the fact that there's a slightly correlation bewtween `Region` and `log_density` and `Area`.

#### AKAIKE INFORMATION CRITERION

In [194]:
zip_model.aic

228288.53644409162

#### ROOT MEAN SQUARE ERROR OF PREDICTION:

In [196]:
y_test, X_test = dmatrices(expr, claimsdf_test, return_type='dataframe')

In [204]:
zip_model_pred = zip_model.predict(X_test, exog_infl=X_test, exposure=claimsdf_test.Exposure)

- Let's create a dataframe with the predctions of the Zip GLM:

In [207]:
zip_pred_df = pd.DataFrame(y_test)
zip_pred_df['predicted_ClaimNb'] = zip_model_pred

In [214]:
rmse_zip = []
for _ in range(10):
    zip_ = zip_pred_df.sample(n=135733)
    rmse_ = ((((zip_["Q('ClaimNb')"] - zip_['predicted_ClaimNb']) ** 2).sum())/13574) ** 0.5
    rmse_zip.append(rmse_)    

In [216]:
zip_rmse = sum(rmse_zip)/len(rmse_zip)
zip_rmse

0.7450743707755763

### ZIP 2

In [ ]:
"""Q('Region_2') + Q('Region_3') + Q('Region_4') + Q('Region_5') + Q('Region_6') + 
 Q('Region_7') + Q('Region_8') + Q('Region_9') + Q('Region_10') + Q('Region_11') +
Q('Region_12') + Q('DrivAge__2') + Q('DrivAge__3') + 
Q('DrivAge__4') + Q('DrivAge__5') + Q('DrivAge__6') + Q('DrivAge__7') + Q('DrivAge__8')"""

In [221]:
expr = """ Q('ClaimNb') ~ Q('Area') + Q('BonusMalus') + Q('log_density') + Q('VehGas_Regular') + 
                          Q('VehPower__5') + Q('VehPower__6') + Q('VehPower__7') + Q('VehPower__8') +
                          Q('VehPower__9') + Q('VehPower__10') + Q('VehPower__11') + Q('VehPower__12') + 
                          Q('VehAge__2') + Q('VehAge__3')
                           """

In [222]:
y_train, X_train = dmatrices(expr, claimsdf_train, return_type='dataframe')

- We define the train sets for the Logistic part of the model

In [223]:
y_train2, X_train2 = dmatrices(expr, claimsdf_train, return_type='dataframe')

In [224]:
X_train2.drop(columns=["Q('Area')", "Q('log_density')", "Q('VehPower__5')", "Q('VehPower__6')", "Q('VehPower__7')", "Q('VehPower__8')", "Q('VehPower__9')", "Q('VehPower__10')", "Q('VehPower__11')", "Q('VehPower__12')"], inplace=True)

In [228]:
zip_model2 = sm.ZeroInflatedPoisson(endog=y_train, exog=X_train, exog_infl=X_train2, exposure=claimsdf_train.Exposure, inflation='logit').fit()

/home/julian/miniconda3/envs/ironcon/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


         Current function value: 0.211026
         Iterations: 35
         Function evaluations: 45
         Gradient evaluations: 45


/home/julian/miniconda3/envs/ironcon/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [226]:
print(zip_model2.summary())

                     ZeroInflatedPoisson Regression Results                    
Dep. Variable:            Q('ClaimNb')   No. Observations:               542280
Model:             ZeroInflatedPoisson   Df Residuals:                   542265
Method:                            MLE   Df Model:                           14
Date:                 Tue, 05 Oct 2021   Pseudo R-squ.:                 0.02340
Time:                         12:54:51   Log-Likelihood:            -1.1444e+05
converged:                       False   LL-Null:                   -1.1718e+05
Covariance Type:             nonrobust   LLR p-value:                     0.000
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
inflate_Intercept               0.4079      0.056      7.249      0.000       0.298       0.518
inflate_Q('BonusMalus')        -0.0141      0.001    -17.704      0.000 

In [247]:
zip_model2.aic

228900.74683430497

#### ROOT MEAN SQUARE ERROR OF PREDICTION:

In [230]:
y_test, X_test = dmatrices(expr, claimsdf_test, return_type='dataframe')

In [244]:
y_test2, X_test2 = dmatrices(expr, claimsdf_test, return_type='dataframe')

In [245]:
X_test2.drop(columns=["Q('Area')", "Q('log_density')", "Q('VehPower__5')", "Q('VehPower__6')", "Q('VehPower__7')", "Q('VehPower__8')", "Q('VehPower__9')", "Q('VehPower__10')", "Q('VehPower__11')", "Q('VehPower__12')"], inplace=True)

In [246]:
zip_model2_pred = zip_model2.predict(X_test, exog_infl=X_test2, exposure=claimsdf_test.Exposure)

- Let's create a dataframe with the predctions of the Zip GLM:

In [249]:
zip2_pred_df = pd.DataFrame(y_test)
zip2_pred_df['predicted_ClaimNb'] = zip_model2_pred

In [250]:
rmse_zip2 = []
for _ in range(10):
    zip2 = zip2_pred_df.sample(n=135733)
    rmse_ = ((((zip2["Q('ClaimNb')"] - zip2['predicted_ClaimNb']) ** 2).sum())/13574) ** 0.5
    rmse_zip2.append(rmse_)    

In [252]:
zip2_rmse = sum(rmse_zip2)/len(rmse_zip2)
zip2_rmse

0.744998169692998

### MODEL SELECTION

- The likelihood ratio test based on Posisson deviance can be applied recursively to a sequence of nested models. This leads to a step-wise reduction of model complexity, this is similar in spirit to the analysis of variance (ANOVA) in Listing 2.7, and it is often referred to as backward model selection